<a href="https://colab.research.google.com/github/MurtazaMister/ML/blob/master/ML_Lab07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import pandas as pd
import io
import matplotlib.pyplot as plt

In [15]:
from google.colab import files
uploaded = files.upload()
data = pd.read_csv(io.BytesIO(uploaded['BuyComputer.csv']))
data.drop(columns=['User ID',],axis=1,inplace=True)
data.head()

Saving BuyComputer.csv to BuyComputer (1).csv


,Age,EstimatedSalary,Purchased
0,19,19000,0
1,35,20000,0
2,26,43000,0
3,27,57000,0
4,19,76000,0


In [16]:
data.shape

(400, 3)

In [17]:
X = data[["Age", "EstimatedSalary"]]
y = data["Purchased"]
X, y

(     Age  EstimatedSalary
 0     19            19000
 1     35            20000
 2     26            43000
 3     27            57000
 4     19            76000
 ..   ...              ...
 395   46            41000
 396   51            23000
 397   50            20000
 398   36            33000
 399   49            36000
 
 [400 rows x 2 columns], 0      0
 1      0
 2      0
 3      0
 4      0
       ..
 395    1
 396    1
 397    1
 398    0
 399    1
 Name: Purchased, Length: 400, dtype: int64)

In [130]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.75)
X_train.shape, X_test.shape, y_train.shape, y_test.shape, X_train.head()

((300, 2), (100, 2), (300,), (100,),      Age  EstimatedSalary
 341   35            75000
 391   47            23000
 350   38           113000
 273   39           106000
 260   35            77000)

In [131]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
y_train, y_test = y_train.tolist(), y_test.tolist()
X_test = sc.transform(X_test)
X_train[:5], y_train[:5]

(array([[-0.26138786,  0.11582583],
        [ 0.88616859, -1.43381654],
        [ 0.02550125,  1.2482568 ],
        [ 0.12113096,  1.03965109],
        [-0.26138786,  0.17542746]]), [0, 1, 1, 1, 0])

In [132]:
import math

def model(x_age, x_salary, theta_0, theta_age, theta_salary):
  return 1 / (1 + math.exp(-1 * (theta_0 + theta_age*x_age + theta_salary*x_salary)))

In [133]:
math.exp(2), math.exp(1)**2, np.exp(1)

(7.38905609893065, 7.3890560989306495, 2.718281828459045)

In [134]:
model(X_train[1][0], X_train[1][1],1,1,1), y[1]

(0.6111983102892117, 0)

In [135]:
import math

def loss(y_pred, y_actual):
  return (y_actual) and (-1 * math.log10(y_pred)) or (-1 * math.log10(1 - y_pred))

In [136]:
loss(model(X_train[0][0], X_train[0][1],1,1,1), y_train[0]), y_train[0], model(X_train[0][0], X_train[0][1],1,1,1)

(0.5250517066240317, 0, 0.7014972796299723)

In [137]:
# Update rule
def update(theta, lr, m, y_pred, y_actual, feature_i=0):
  summation = 0
  for i in range(len(y_pred)):
    summation = summation + (y_pred[i] - y_actual[i])*(feature_i and feature_i[i] or 1)
  # print("summ: ",summation)
  return theta - (lr/m)*summation

In [156]:
# For 1 epoch
# We have model, loss fn, and update rule
theta_0, theta_age, theta_salary = 1,1,1
epochs = 10 # 100
learning_rate = 10

X_train_df = pd.DataFrame(X_train, columns=["Age", "EstimatedSalary"])
age = X_train_df["Age"].tolist()
estimated_salary = X_train_df["EstimatedSalary"].tolist()

for j in range(epochs):
  y_train_pred = []
  m = len(X_train)
  for i in range(m):
    y_train_pred.append(model(X_train[i][0], X_train[i][1], theta_0, theta_age, theta_salary))
    loss_value = loss(y_train_pred[i], y_train[i])
    # print(f"Epoch {j} Iteration {i+1}: Loss: ",loss_value)
  theta_0 = update(theta_0, learning_rate, m, y_train_pred, y_train)
  theta_age = update(theta_age, learning_rate, m, y_train_pred, y_train, age)
  theta_salary = update(theta_salary, learning_rate, m, y_train_pred, y_train, estimated_salary)
  # print("New theta values: ",theta_0, theta_age, theta_salary)

In [160]:
y_test_pred = []
for i in range(len(X_test)):
  if model(X_test[i][0], X_test[i][1], theta_0, theta_age, theta_salary) < 0.5:
    y_test_pred.append(0)
  else:
    y_test_pred.append(1)

accuracy = 0 # (TP + TN)/Total
precision = 0 # (TP/TP+FP)
recall = 0 # (TP/TP+FN)
# F_Score = 2PR/(P+R)
TP, FP, TN, FN = 0,0,0,0

for i in range(len(X_test)):
  if y_test_pred[i]==1 and y_test[i]==1:
    TP = TP+1
  if y_test[i]==0 and y_test_pred[i]==1:
    FP = FP+1
  if y_test_pred[i]==0 and y_test[i]==0:
    TN = TN+1
  if y_test_pred[i]==0 and y_test[i]==1:
    FN = FN+1
    
accuracy = (TP+TN)/len(X_test)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
F_Score = (2*precision*recall)/(precision+recall)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF Score: {F_Score}")

Accuracy: 0.93
Precision: 0.9629629629629629
Recall: 0.8125
F Score: 0.8813559322033898
